In [1]:
# Python Imports
import ipywidgets as widgets
import networkx as nx
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

# Project Imports
from src.mesh_analysis.readers.file_reader import FileReader
from src.mesh_analysis.tracers.waku_tracer import WakuTracer

In [27]:
tracer = WakuTracer()
reader = FileReader("../../../test/log_test/", tracer)

df = reader.read()
test = df.reset_index()

In [59]:
%matplotlib
dropdown1 = widgets.SelectMultiple(
                        options=df.index.get_level_values(0).drop_duplicates(),
                        description='Timestamp',
                        disabled=False,
                        layout={'height':'100px', 'width':'40%'})
dropdown2 = widgets.SelectMultiple(
                        options=df.index.get_level_values(1).drop_duplicates(),
                        description='Msg hash',
                        disabled=False,
                        layout={'height':'100px', 'width':'40%'})
    
fig = plt.figure()
ax_ = fig.add_subplot(1, 1, 1)
def test(x, y, data_):
    ax_.cla()
    first_timestamp = data_.index.get_level_values(0)[0]
    df_ = data_.loc[(first_timestamp,y[0]):(x[0], y[0])]
    df_.reset_index(inplace=True)
    G = nx.from_pandas_edgelist(df_, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash'], create_using=nx.DiGraph)
    timestamp_edges = nx.get_edge_attributes(G, 'timestamp')
    hash_edges = nx.get_edge_attributes(G, 'msg_hash')
    ds = [timestamp_edges, hash_edges]
    d = {}
    for k in timestamp_edges.keys():
        d[k] = tuple(d[k] for d in ds)
    pos = nx.spring_layout(G)
    nx.draw(G, pos, ax=ax_, with_labels=True)
    nx.draw_networkx_edge_labels(G, pos, ax=ax_, edge_labels=d)
    plt.show()


i_plot = widgets.interactive(test, x=dropdown1, y=dropdown2, data_=widgets.fixed(df))
i_plot

Using matplotlib backend: QtAgg


interactive(children=(SelectMultiple(description='Timestamp', layout=Layout(height='100px', width='40%'), opti…

'2024-04-22 14:06:58.001+00:00'